In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/Groceries data.csv')

In [ ]:
df.head()

,Member_number,Date,itemDescription,year,month,day,day_of_week
0,1808,2015-07-21,tropical fruit,2015,7,21,1
1,2552,2015-05-01,whole milk,2015,5,1,4
2,2300,2015-09-19,pip fruit,2015,9,19,5
3,1187,2015-12-12,other vegetables,2015,12,12,5
4,3037,2015-01-02,whole milk,2015,1,2,4


In [ ]:
#number of unique customer
unique_members = df['Member_number'].nunique()
print(f"The number of unique member numbers is: {unique_members}")

The number of unique member numbers is: 3898


In [ ]:
product_dimension = pd.DataFrame(df['itemDescription'].unique(), columns=['product_name'])
product_dimension['product_id'] = range(1, len(product_dimension) + 1)
print(product_dimension)

              product_name  product_id
0           tropical fruit           1
1               whole milk           2
2                pip fruit           3
3         other vegetables           4
4               rolls/buns           5
..                     ...         ...
162         pudding powder         163
163            ready soups         164
164        make up remover         165
165         toilet cleaner         166
166  preservation products         167

[167 rows x 2 columns]


In [ ]:
product_dimension.head()

,product_name,product_id
0,tropical fruit,1
1,whole milk,2
2,pip fruit,3
3,other vegetables,4
4,rolls/buns,5


In [ ]:
date_dimension = pd.DataFrame(df['Date'].unique(), columns=['date'])
date_dimension['date_id'] = range(1, len(date_dimension) + 1)
date_dimension['year'] = pd.to_datetime(date_dimension['date']).dt.year
date_dimension['month'] = pd.to_datetime(date_dimension['date']).dt.month
date_dimension['day'] = pd.to_datetime(date_dimension['date']).dt.day
print(date_dimension)

           date  date_id  year  month  day
0    2015-07-21        1  2015      7   21
1    2015-05-01        2  2015      5    1
2    2015-09-19        3  2015      9   19
3    2015-12-12        4  2015     12   12
4    2015-01-02        5  2015      1    2
..          ...      ...   ...    ...  ...
723  2014-11-02      724  2014     11    2
724  2014-09-15      725  2014      9   15
725  2014-10-02      726  2014     10    2
726  2014-06-29      727  2014      6   29
727  2014-12-11      728  2014     12   11

[728 rows x 5 columns]


In [ ]:
date_dimension.head()

,date,date_id,year,month,day
0,2015-07-21,1,2015,7,21
1,2015-05-01,2,2015,5,1
2,2015-09-19,3,2015,9,19
3,2015-12-12,4,2015,12,12
4,2015-01-02,5,2015,1,2


In [ ]:
fact_table = df.merge(product_dimension, left_on='itemDescription', right_on='product_name') \
               .merge(date_dimension, left_on='Date', right_on='date')
fact_table = fact_table[['Member_number', 'date_id', 'product_id']]
fact_table.rename(columns={'Member_number': 'customer_id'}, inplace=True)
print(fact_table)

       customer_id  date_id  product_id
0             1808        1           1
1             2552        2           2
2             2300        3           3
3             1187        4           4
4             3037        5           2
...            ...      ...         ...
38760         4471      396          76
38761         2022      427          65
38762         1097      625         154
38763         1510      713          12
38764         1521      556         104

[38765 rows x 3 columns]


In [ ]:
fact_table.head()

,customer_id,date_id,product_id
0,1808,1,1
1,2552,2,2
2,2300,3,3
3,1187,4,4
4,3037,5,2


In [ ]:
# prompt: customer_1808 = fact_table[fact_table['customer_id'] == 1808] sort by date_id

customer_1808 = fact_table[fact_table['customer_id'] == 1808].sort_values(by='date_id')
customer_1808

,customer_id,date_id,product_id
0,1808,1,1
16149,1808,1,5
36088,1808,1,65
4355,1808,225,119
20504,1808,225,103
11488,1808,452,35
27637,1808,452,7
38731,1808,452,114
9090,1808,472,57
25239,1808,472,2


In [ ]:
# Create transaction ID
fact_table['transaction_id'] = fact_table.groupby(['customer_id', 'date_id']).ngroup() + 1

customer_1808 = fact_table[fact_table['customer_id'] == 1808].sort_values(by='date_id')
customer_1808

,customer_id,date_id,product_id,transaction_id
0,1808,1,1,2985
16149,1808,1,5,2985
36088,1808,1,65,2985
4355,1808,225,119,2986
20504,1808,225,103,2986
11488,1808,452,35,2987
27637,1808,452,7,2987
38731,1808,452,114,2987
9090,1808,472,57,2988
25239,1808,472,2,2988


In [ ]:

# Get transaction with ID 2870
transaction_2870 = fact_table[fact_table['transaction_id'] == 2870]
transaction_2870

,customer_id,date_id,product_id,transaction_id
2180,1780,167,3,2870
6706,1780,167,45,2870
7557,1780,167,15,2870
7911,1780,167,49,2870
8513,1780,167,19,2870
18329,1780,167,128,2870
22855,1780,167,5,2870
23706,1780,167,18,2870
24060,1780,167,4,2870
24662,1780,167,45,2870


In [ ]:
customer_dimension = pd.DataFrame(df['Member_number'].unique(), columns=['customer_id'])


# Display the customer dimension table
customer_dimension.head()

,customer_id
0,1808
1,2552
2,2300
3,1187
4,3037


In [ ]:
item_counts = df['itemDescription'].value_counts()
item_counts

,count
itemDescription,
whole milk,2502
other vegetables,1898
rolls/buns,1716
soda,1514
yogurt,1334
...,...
rubbing alcohol,5
bags,4
baby cosmetics,3


In [ ]:
import pandas as pd

# Load the CSV file
file_path = "/content/customer_dim.csv"  # Replace with the path to your CSV file
table_name = "customer_dim"       # Replace with the name of your table
df = pd.read_csv(file_path)

# Generate INSERT statements
insert_statements = []
for index, row in df.iterrows():
    # Convert row values to a string, escaping single quotes
    values = ', '.join([f"'{value}'" if isinstance(value, str) else str(value) for value in row.values])
    insert_statements.append(f"INSERT INTO {table_name} VALUES ({values});")

# Write to a file (optional)
output_file = "insert_statements.sql"
with open(output_file, "w") as f:
    f.write("\n".join(insert_statements))

print(f"INSERT statements written to {output_file}")

INSERT statements written to insert_statements.sql


In [ ]:
import pandas as pd

# Load the CSV file
file_path = "/content/fact_table.csv"  # Replace with the path to your CSV file
table_name = "fact_table"       # Replace with the name of your table
df = pd.read_csv(file_path)

# Generate INSERT statements
insert_statements = []
for index, row in df.iterrows():
    # Convert row values to a string, escaping single quotes
    values = ', '.join([f"'{value}'" if isinstance(value, str) else str(value) for value in row.values])
    insert_statements.append(f"INSERT INTO {table_name} VALUES ({values});")

# Write to a file (optional)
output_file = "insert_statements_fact.sql"
with open(output_file, "w") as f:
    f.write("\n".join(insert_statements))

print(f"INSERT statements written to {output_file}")

INSERT statements written to insert_statements_fact.sql


In [ ]:
fact_table.head()

,customer_id,date_id,product_id,transaction_id
0,1808,1,1,2985
1,2552,2,2,5780
2,2300,3,3,4849
3,1187,4,4,686
4,3037,5,2,7585


In [ ]:
fact_table.to_csv('fact_table.csv', index=False)

In [ ]:
import pandas as pd

# Load the CSV file
file_path = "/content/product_dim.csv"  # Replace with the path to your CSV file
table_name = "product_dim"       # Replace with the name of your table
df = pd.read_csv(file_path)

# Generate INSERT statements
insert_statements = []
for index, row in df.iterrows():
    # Convert row values to a string, escaping single quotes
    values = ', '.join([f"'{value}'" if isinstance(value, str) else str(value) for value in row.values])
    insert_statements.append(f"INSERT INTO {table_name} VALUES ({values});")

# Write to a file (optional)
output_file = "insert_statements_product_dim.sql"
with open(output_file, "w") as f:
    f.write("\n".join(insert_statements))

print(f"INSERT statements written to {output_file}")

INSERT statements written to insert_statements_product_dim.sql


In [ ]:
# prompt: read from product_dim csv and head

import pandas as pd
product_dimension = pd.read_csv('/content/product_dim.csv')
product_dimension.head()

,product_name,product_id,category,subcategory,price
0,tropical fruit,1,Fruit,Fruit,1.20
1,whole milk,2,Dairy,Milk,1.70
2,pip fruit,3,Fruit,Fruit,2.79
3,other vegetables,4,Vegetables,Vegetable,2.01
4,rolls/buns,5,Bakery,Roll,1.52


In [ ]:
file_path = "/content/date_dim.csv"  # Replace with the path to your CSV file
table_name = "date_dim"       # Replace with the name of your table
df = pd.read_csv(file_path)

# Generate INSERT statements
insert_statements = []
for index, row in df.iterrows():
    # Convert date to a string with the correct format for your database
    date_str = row['date']
    # Assuming the 'date' column is already in a compatible date format
    day_of_week = pd.to_datetime(date_str).day_name()
    is_weekend = pd.to_datetime(date_str).dayofweek > 4
    week_of_year = pd.to_datetime(date_str).weekofyear

    insert_statements.append(f"INSERT INTO {table_name} (date, date_id, year, month, day, is_weekend, day_of_week, week_of_year) VALUES (TO_DATE('{date_str}', 'YYYY-MM-DD'), {row['date_id']}, {row['year']}, {row['month']}, {row['day']}, {is_weekend}, '{day_of_week}', {week_of_year});")

# Write to a file (optional)
output_file = "insert_statements_date_dim3.sql"
with open(output_file, "w") as f:
    f.write("\n".join(insert_statements))

print(f"INSERT statements written to {output_file}")

INSERT statements written to insert_statements_date_dim3.sql


In [ ]:
date_dimension = pd.read_csv('/content/date_dim.csv')
date_dimension.head()

,date,date_id,year,month,day
0,2015-07-21,1,2015,7,21
1,2015-05-01,2,2015,5,1
2,2015-09-19,3,2015,9,19
3,2015-12-12,4,2015,12,12
4,2015-01-02,5,2015,1,2


In [ ]:
# prompt: read product dim

import pandas as pd
product_dimension = pd.read_csv('/content/product_dim.csv')
product_dimension

,product_name,product_id,price,category_id,subcategory_id
0,tropical fruit,1,1.20,1,1
1,pip fruit,3,2.79,1,1
2,citrus fruit,7,1.85,1,1
3,fruit/vegetable juice,12,1.69,1,1
4,packaged fruit/vegetables,13,1.63,1,1
...,...,...,...,...,...
162,mayonnaise,129,1.64,25,79
163,organic products,153,1.54,25,92
164,flower soil/fertilizer,133,1.84,26,82
165,seasonal products,142,2.18,27,87


In [ ]:
# prompt: create insert statement in oracle from product_dim file with all columns as product
# name, id, price, cate and sub cat

import pandas as pd

# Load the CSV file
file_path = "/content/product_dim.csv"  # Replace with the path to your CSV file
table_name = "product_dim"       # Replace with the name of your table
df = pd.read_csv(file_path)
df['category_id'] = df['category_id'].astype(int)
df['subcategory_id'] = df['subcategory_id'].astype(int)
# Generate INSERT statements
insert_statements = []
for index, row in df.iterrows():
    # Convert row values to a string, escaping single quotes
    product_name = row['product_name'].replace("'", "''")  # Escape single quotes in product_name
    product_id = row['product_id']
    # Add placeholders for price, category, and subcategory if they don't exist
    price = row['price']
    category = row['category_id']
    subcategory = row['subcategory_id']

    insert_statements.append(f"INSERT INTO {table_name} (product_name, product_id, price, category, subcategory) VALUES ('{product_name}', {product_id}, {price}, {category}, {subcategory});")


# Write to a file (optional)
output_file = "insert_statements_product_dim_final.sql"
with open(output_file, "w") as f:
    f.write("\n".join(insert_statements))

print(f"INSERT statements written to {output_file}")

INSERT statements written to insert_statements_product_dim_final.sql


In [ ]:
# prompt: make insert statement from categories_dim with column category and category_id

# Load the CSV file
file_path = "/content/categories_dim.csv"  # Replace with the actual file path
table_name = "categories_dim"  # Replace with your table name
df = pd.read_csv(file_path)

# Generate INSERT statements
insert_statements = []
for index, row in df.iterrows():
    category = row['category'].replace("'", "''")  # Escape single quotes
    category_id = row['category_id']
    insert_statements.append(f"INSERT INTO {table_name} (category, category_id) VALUES ('{category}', {category_id});")

# Write to a file (optional)
output_file = "insert_statements_categories_dim.sql"
with open(output_file, "w") as f:
    f.write("\n".join(insert_statements))

print(f"INSERT statements written to {output_file}")

INSERT statements written to insert_statements_categories_dim.sql


In [ ]:
# prompt: make insert statement from subcategories_dim with column subcategory and subcategory_id

# Load the CSV file
file_path = "/content/subcategories_dim.csv"  # Replace with the actual file path
table_name = "subcategories_dim"  # Replace with your table name
df = pd.read_csv(file_path)

# Generate INSERT statements
insert_statements = []
for index, row in df.iterrows():
    subcategory = row['subcategory'].replace("'", "''")  # Escape single quotes
    subcategory_id = row['subcategory_id']
    insert_statements.append(f"INSERT INTO {table_name} (subcategory, subcategory_id) VALUES ('{subcategory}', {subcategory_id});")

# Write to a file (optional)
output_file = "insert_statements_subcategories_dim.sql"
with open(output_file, "w") as f:
    f.write("\n".join(insert_statements))

print(f"INSERT statements written to {output_file}")

INSERT statements written to insert_statements_subcategories_dim.sql


In [ ]:
# prompt: change date format to 'YYYY-MM-DD'

date_dimension['date'] = pd.to_datetime(date_dimension['date']).dt.strftime('%Y-%m-%d')
date_dimension

,date,date_id,year,month,day
0,2015-07-21,1,2015,7,21
1,2015-05-01,2,2015,5,1
2,2015-09-19,3,2015,9,19
3,2015-12-12,4,2015,12,12
4,2015-01-02,5,2015,1,2
...,...,...,...,...,...
723,2014-11-02,724,2014,11,2
724,2014-09-15,725,2014,9,15
725,2014-10-02,726,2014,10,2
726,2014-06-29,727,2014,6,29


In [ ]:
# prompt: read the fact_table

import pandas as pd
fact_table = pd.read_csv('/content/fact_table.csv')
fact_table.head()

,customer_id,date_id,product_id,transaction_id
0,1808,1,1,2985
1,2552,2,2,5780
2,2300,3,3,4849
3,1187,4,4,686
4,3037,5,2,7585
